In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-02 2021-12-03


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|██████████████████▍                                                                                                | 81/505 [01:00<05:00,  1.41it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|█████████████████████████████████████████████████████████████████████                                             | 306/505 [03:48<02:29,  1.33it/s]

project_market prices documents must be a non-empty list


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 502/505 [06:17<00:02,  1.36it/s]

...

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 504/505 [06:18<00:00,  1.38it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [06:19<00:00,  1.33it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|██████████████▊                                                                                                    | 65/505 [02:02<13:49,  1.89s/it]

'date'


 16%|█████████████████▉                                                                                                 | 79/505 [02:29<13:16,  1.87s/it]

'date'


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 502/505 [15:50<00:05,  1.86s/it]

...

'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 504/505 [15:54<00:01,  1.84s/it]

'date'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [15:56<00:00,  1.89s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
274,0.410903,0.140777,0.110420,0.054309,0.110420,KR,Kroger,Consumer Staples
137,-0.053963,-0.192596,0.092753,0.011038,0.092753,DAL,Delta Air Lines,Industrials
85,-0.122179,-0.314297,0.092186,-0.014868,0.092186,CCL,Carnival Corporation,Consumer Discretionary
80,0.257402,-0.246548,0.083785,-0.036906,0.083785,CZR,Caesars Entertainment,Consumer Discretionary
494,-0.230215,-0.056416,0.081909,-0.008554,0.081909,WYNN,Wynn Resorts,Consumer Discretionary
341,-0.192583,-0.323207,0.077010,-0.052423,0.077010,NCLH,Norwegian Cruise Line Holdings,Consumer Discretionary
310,0.388469,-0.080918,0.076501,0.002431,0.076501,MGM,MGM Resorts International,Consumer Discretionary
69,-0.001677,-0.104513,0.075403,0.019547,0.075403,BA,Boeing Company,Industrials
227,0.298995,0.010423,0.074100,0.011009,0.074100,HLT,Hilton Worldwide,Consumer Discretionary
405,-0.026018,-0.253980,0.071729,-0.014451,0.071729,RCL,Royal Caribbean Group,Consumer Discretionary


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Financials,0.348692,0.008908,0.032914,-0.010507,0.032914
Real Estate,0.435258,0.054044,0.032227,-0.008846,0.032227
Consumer Discretionary,0.261836,0.036838,0.032050,-0.014186,0.032050
Energy,0.576269,0.014378,0.030725,-0.008717,0.031749
Industrials,0.244975,0.032666,0.030129,-0.014536,0.030129
Communication Services,0.077604,-0.103068,0.024856,-0.031935,0.024856
Materials,0.242515,0.039890,0.022671,-0.023760,0.022671
Information Technology,0.238502,0.048416,0.017748,-0.017092,0.017748
Consumer Staples,0.058544,0.005797,0.016363,-0.018400,0.016363
